In [1]:
import datetime
import json
import os

import markdown
import pandas as pd
import pymysql
import requests
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
version = "2024-08-25"

In [2]:
# college_postid=9228
# college_name="加州大学洛杉矶分校"
connection = pymysql.connect(
    db=os.environ["db_name"],
    user=os.environ["db_user"],
    passwd=os.environ["db_pass"],
    host=os.environ["db_host"],
    port=3306,
    cursorclass=pymysql.cursors.DictCursor,
)
cursor = connection.cursor()
query = """SELECT ca,efa,efb,ranking FROM fp_IPEDS.latest_information"""
cursor.execute(query)
row = cursor.fetchone()
# connection.commit()
rank_year = row["ranking"]
major_year = row["ca"]
students_year = row["efa"] + 1
age_year = row["efb"] + 1
cursor.close()
connection.close()

In [3]:
comment_string = """对该数据做出一段300字左右文字的简短的评论，标题层级不要超过一级，如果要在输出内容中使用标题的，请使用h3标题，比如<h3>...</h3>。\
可以结合你知道的其他关于该类数据的信息一起评论，可以结合真实的新闻时事信息，但不要捏造信息，遇到比率等需要转换成百分比数。直接给出评论内容，在回答中不要提及数据项的代号，只使用各项数据的真实名称。\
数据中有数值的都为已经发布的实际数据，数据中不含任何预计数据，未发布数据的为'NaN'或者空位，这部分可以忽略。Final output should be in purely English language"""
rank_admin_data_url = "https://www.forwardpathway.com/d3v7/dataphp/school_database/ranking_admin_20231213.php?name="
information_data_url = "https://www.forwardpathway.com/d3v7/dataphp/school_database/school_information_20240821.php?name="
majors_data_url = (
    "https://www.forwardpathway.com/d3v7/dataphp/chatbot/degrees.php?name="
)
SAT_data_url = "https://www.forwardpathway.com/d3v7/dataphp/school_database/score10_20231213.php?name="
students_data_url = "https://www.forwardpathway.com/d3v7/dataphp/school_database/student_comp_20240118.php?name="
age_data_url = "https://www.forwardpathway.com/d3v7/dataphp/school_database/age_mf_20240118.php?name="
international_data_url = "https://www.forwardpathway.com/d3v7/dataphp/school_database/international_students_20240118.php?name="
crime_data_url = "https://www.forwardpathway.com/d3v7/dataphp/school_database/crime_yearly_20240324.php?name="
nearby_data_url = (
    "https://www.forwardpathway.com/d3v7/dataphp/chatbot/school_nearby.php?name="
)

---------------排名录取率--------------------

In [4]:
def rank_admin_comments(college_postid, college_name):
    rank_admin_data = pd.read_json(rank_admin_data_url + str(college_postid))
    rank_admin_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """下面给出的是"""
                + college_name
                + """排名与录取相关数据，数据中'year'为年份，'rank'为USNewws美国大学排名，'rate'为男生录取率,'rate2'为女生录取率,'enroll'为当年入学人数,'deny'为当年拒绝人数,'defer'为当年录取但延迟入学人数，
                        """
                + comment_string,
            ),
            ("human", "排名与录取数据如下：{data}"),
        ]
    )
    rank_admin_chain = rank_admin_prompt | llm | StrOutputParser()
    response = rank_admin_chain.invoke({"data": rank_admin_data}).replace("\n\n", "\n")
    rank_admin_comments = markdown.markdown(response)
    return rank_admin_comments

------------------------基础数据-------------------------------

In [5]:
def information_comments(college_postid, college_name):
    # information_data=pd.read_json(information_data_url+str(college_postid))
    reqeust_response = requests.get(information_data_url + str(college_postid))
    information_data = json.loads(reqeust_response.content)
    information_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """下面给出的是"""
                + college_name
                + """的相关数据，数据中'type'为学校类型，1为公立，2为私立
                        'tuition'为学费数据，其中'year'为年份，'tuition_in_under'为州内本科学费,'tuition_out_under'为外州本科生学费，'tuition_in_grad'为州内研究生学费，'tuition_out_grad'为外州研究生学费；
                        'students'为学生人数数据，其中'year'为年份，'students_under'为该校本科生人数，'students_grad'为该校研究生人数；
                        'm2w'为男女比例数据，其中'year'为年份，'m2w_men_under'为本科生男生占比，'m2w_women_under'为本科生女生占比，'m2w_men_grad'为研究生男生占比，'m2w_women_grad'为研究生女生占比；
                        'graduation'为毕业率数据，其中'year'为年份，'graduation_100_under'为本科生按时毕业率，'graduation_150_under'为本科生150%时间毕业率；
                        'retention'为学生保有率数据，其中'year'为年份，'retention_under'为学生保有率；
                        's2f'为学生教授比，其中'year'为年份，'s2f_under'为学生教授比；
                        选取其中最重要或者变化比较大3-4个数据做讨论，不一定要所有点都提到"""
                + comment_string,
            ),
            ("human", "数据如下：{data}"),
        ]
    )
    information_chain = information_prompt | llm | StrOutputParser()
    response = information_chain.invoke({"data": information_data})
    information_comments = markdown.markdown(response).replace("\n", "")
    return information_comments

-----------------------------专业数据---------------------------------

In [6]:
def majors_comments(college_postid, college_name):
    majors_data = pd.read_json(
        majors_data_url + str(college_postid), orient="index", convert_axes=False
    ).reset_index(drop=True)
    majors_data = majors_data.iloc[
        majors_data[["bachelor", "master", "doctorate"]]
        .sum(axis=1)
        .sort_values(ascending=False)
        .index
    ]
    majors_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """下面给出的是"""
                + str(major_year)
                + """年"""
                + college_name
                + """的专业毕业人数相关数据，其中associate为副学士学位，bachelor为本科学位，master为硕士学位，doctorate为博士学位，
                        associate_i为副学士学位国际留学生，bachelor_i为本科学位国际留学生，master_i为硕士学位国际留学生，doctorate_i为博士学位国际留学生，
                        学位后面的数字为毕业年份，该栏对应数字为该专业该年份毕业的人数，
                        从数据中找几个重要的或者有特点的类别分析，"""
                + comment_string,
            ),
            ("human", "数据如下：{data}"),
        ]
    )
    majors_chain = majors_prompt | llm | StrOutputParser()
    response = majors_chain.invoke({"data": majors_data})
    majors_comments = markdown.markdown(response).replace("\n", "")
    return majors_comments

----------------------------------SAT/ACT--------------------------------------

In [7]:
def SAT_comments(college_postid, college_name):
    SAT_data = pd.read_json(SAT_data_url + str(college_postid))
    SAT_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """下面给出的是"""
                + college_name
                + """的相关数据，第一层为不同年份的数据，下面分为per和score部分，
            其中per部分包含SAT和ACT成绩提交的人数（'number'）和占录取人数的比例（'per'），
            score部分包含'SATR'为SAT阅读成绩，'SATM'为SAT数学成绩，'ACTC'为ACT综合成绩，'ACTE'为ACT英语成绩，'ACTM'为ACT数学成绩，
            'start'是录取学生中25%该项成绩，'end'是录取学生总75%该项成绩，"""
                + comment_string,
            ),
            ("human", "数据如下：{data}"),
        ]
    )
    SAT_chain = SAT_prompt | llm | StrOutputParser()
    response = SAT_chain.invoke({"data": SAT_data})
    SAT_comments = markdown.markdown(response).replace("\n", "")
    return SAT_comments

------------------------学生组成---------------------------------

In [8]:
def students_comments(college_postid, college_name):
    students_data = pd.read_json(students_data_url + str(college_postid))
    students_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """下面给出的是"""
                + str(students_year)
                + "年"
                + college_name
                + """的学生组成相关数据，其中'name'是学生分类，有以下几种：'uf'是本科新生，'uj'是本科老生，'ut'是本科转学生，'gr'是研究生，
            'value'是该类学生的人数，'ratioM'是该类学生男生占比，'ratioW'是该类学生女生占比，第一项的'year'是数据的年份
            subs是该类学生的细分人种，有'wh'是白人，'as'是亚裔，'la'是拉丁裔，'pa'是太平洋岛国及其他，'af'是非裔，'nr'是留学生，学生人数代码依旧为'value'，男女比例代码也与上一层相同，在回答中请不要提及各个分类的代码，仅使用真实名称，"""
                + comment_string,
            ),
            ("human", "数据如下：{data}"),
        ]
    )
    students_chain = students_prompt | llm | StrOutputParser()
    response = students_chain.invoke({"data": students_data})
    students_comments = markdown.markdown(response).replace("\n", "")
    # students_comments=response
    return students_comments

-------------------学生年龄分布--------------------------------

In [9]:
def age_comments(college_postid, college_name):
    age_data = pd.read_json(age_data_url + str(college_postid))
    age_data[["underf", "gradf", "totalf"]] = -age_data[["underf", "gradf", "totalf"]]
    age_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """下面给出的是"""
                + str(age_year)
                + "年"
                + college_name
                + """的学生年龄组成相关数据，其中'cat'为不同年龄段的分类，'underm'是本科男生，'underf'是本科女生，'gradm'是研究生男生，'gradf'是研究生女生，
            'totalm'是所有学生的那声，'totalf'是所有学生的女生，"""
                + comment_string,
            ),
            ("human", "数据如下：{data}"),
        ]
    )
    age_chain = age_prompt | llm | StrOutputParser()
    response = age_chain.invoke({"data": age_data})
    age_comments = markdown.markdown(response).replace("\n", "")
    return age_comments

-------------------国际学生数量-----------------------------

In [10]:
def international_comments(college_postid, college_name):
    international_data = pd.read_json(international_data_url + str(college_postid))
    international_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """下面给出的是"""
                + college_name
                + """的国际留学生相关数据，'year'为数据年份，其中'undertotal'为该校本科总学生数量，'under'为留学生数量，'underper'为本科学生中留学生占比，
            'gradtotal'为该校研究生总学生数量，'grad'为研究生中留学生数量，'gradper'为研究生中留学生占比"""
                + comment_string,
            ),
            ("human", "数据如下：{data}"),
        ]
    )
    international_chain = international_prompt | llm | StrOutputParser()
    response = international_chain.invoke({"data": international_data})
    international_comments = markdown.markdown(response).replace("\n", "")
    return international_comments

-------------------------犯罪率---------------------------------------

In [11]:
def crime_comments(college_postid, college_name):
    crime_data = pd.read_json(crime_data_url + str(college_postid))
    crime_dict = {
        "WA": "持枪逮捕",
        "DA": "毒品逮捕",
        "LA": "酗酒逮捕",
        "WD": "持枪记过",
        "DD": "毒品记过",
        "LD": "酗酒记过",
        "DoV": "家暴",
        "DaV": "约会犯罪",
        "S": "跟踪",
        "M": "谋杀",
        "NM": "过失杀人",
        "Ra": "强奸",
        "F": "性扰",
        "I": "乱伦",
        "Ro": "抢劫",
        "AA": "袭击",
        "B": "盗窃",
        "VT": "偷车",
        "A": "纵火",
        "RHF": "宿舍火灾",
    }
    crime_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """下面给出的是"""
                + college_name
                + """的校园记过、犯罪相关数据，'year'为每组数据的年份，'crime_total'可以忽略，'avg1000'为每1000学生记过、犯罪率，
            'subdata'为细分项，项目名称代码转换为真实名称如下表：{crime_dict}，其中'numberR'为数量，忽略'number'项，在回答最后附上犯罪率的计算方法链接：'https://www.forwardpathway.com/33447'，"""
                + comment_string,
            ),
            ("human", "数据如下：{data}"),
        ]
    )
    crime_chain = crime_prompt | llm | StrOutputParser()
    response = crime_chain.invoke({"data": crime_data, "crime_dict": crime_dict})
    crime_comments = markdown.markdown(response).replace("\n", "")
    return crime_comments

------------------------周边名校------------------------------------

In [12]:
def nearby_comments(college_postid, college_name):
    nearby_data = pd.read_json(nearby_data_url + str(college_postid))
    nearby_data = (
        nearby_data.sort_values("distance").drop(columns=["distance"]).head(20)
    )
    nearby_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """下面给出的是"""
                + college_name
                + """的周边名校，数据中'name'为英文名，'cname'为中文名，'rank'为"""
                + str(rank_year)
                + """USNews排名，'type'为学校类型，1为综合大学，2为文理学院，
            对该校周边名校中挑选重点做一个简单的介绍，提到的学校需要链接，链接名为学校名称，链接url为'https://www.forwardpathway.com/'+'postid'，final ouput should be purely in English language""",
            ),
            ("human", "数据如下：{data}"),
        ]
    )
    nearby_chain = nearby_prompt | llm | StrOutputParser()
    response = nearby_chain.invoke({"data": nearby_data})
    nearby_comments = markdown.markdown(response).replace("\n", "")
    return nearby_comments

----------------------------简介--------------------------------------

In [13]:
def intro_info(college_cname, college_ename):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """对下面给出的学校中文名和英文名，结合你了解的情况，对该学校做一个500字左右最多两个段落的简单综合介绍，可以从下面的几个方面随机选取3-4个方面介绍学校，可以选择的方面如下：特点、校训、人文环境、校园文化、学校设施、师资配置、地理位置、毕业生就业、专业、体育运动、学生生活、杰出校友、最近的热点新闻等等，内容不可以虚构必须是事实，内容尽量不要提及任何负面信息，final output should purely in English""",
            ),
            ("human", "中文名如下：{college_cname}，英文名如下：{college_ename}"),
        ]
    )
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke(
        {"college_cname": college_cname, "college_ename": college_ename}
    )
    comments = markdown.markdown(response).replace("\n", "")
    return comments

-------------------------------写入database-----------------------------------

In [14]:
connection = pymysql.connect(
    db=os.environ["db_name"],
    user=os.environ["db_user"],
    passwd=os.environ["db_pass"],
    host=os.environ["db_host"],
    port=3306,
    cursorclass=pymysql.cursors.DictCursor,
)
cursor = connection.cursor()
query = """SELECT t1.postid,t2.postid as fpus_id,t1.unitid,t1.name,t1.cname,t1.type FROM fp_ranking.colleges t1 LEFT JOIN fpus_colleges.transform t2 ON t2.unitid=t1.unitid WHERE t1.type IN (1,2)"""
cursor.execute(query)
rows = cursor.fetchall()
colleges = pd.DataFrame(
    columns=["postid", "fpus_id", "unitid", "name", "cname", "type"]
)
for row in rows:
    colleges = pd.concat([colleges, pd.DataFrame([row])])
colleges = colleges.reset_index(drop=True)
cursor.close()
connection.close()

In [15]:
connection = pymysql.connect(
    db=os.environ["db_name"],
    user=os.environ["db_user"],
    passwd=os.environ["db_pass"],
    host=os.environ["db_host"],
    port=3306,
)
cursor = connection.cursor()

In [16]:
version = "2024-08-21"
update_sections = {
    "rank_admin": rank_admin_comments,
    "information": information_comments,
    "majors": majors_comments,
    "SAT_ACT": SAT_comments,
    "stduents_comp": students_comments,
    "age": age_comments,
    "international": international_comments,
    "crime": crime_comments,
    "nearby": nearby_comments,
    "intro": intro_info,
}
update_sections = {
    "intro": intro_info,
    "information": information_comments,
    "majors": majors_comments,
}
n = 0
for index, row in colleges.iterrows():
    print(n, end="-")
    postid = row["postid"]
    fpus_id=row["fpus_id"]
    unitid = row["unitid"]
    college_name = row["name"]
    college_cname = row["cname"]
    for update_key in update_sections:
        query = "INSERT INTO fp_chatGPT.data_comments_en (postid,unitid,type,comment,version) VALUES (%s,%s,%s,%s,%s)"
        cursor.execute(
            query,
            (
                fpus_id,
                unitid,
                update_key,
                update_sections[update_key](postid, college_name),
                version,
            ),
        )
    utc_time_now = datetime.datetime.now(datetime.UTC).strftime("%Y-%m-%d %H:%M:%S")
    time_now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    query = "UPDATE fpus_wordpress.`wp_posts` SET `post_modified`=%s,`post_modified_gmt`=%s WHERE `ID`=%s"
    cursor.execute(query, (time_now, utc_time_now, fpus_id))
    connection.commit()
    n = n + 1
    if n % 10 == 0:
        print(" ")
cursor.close()
connection.close()

0-1-2-3-4-5-6-7-8-9- 
10-11-12-13-14-15-16-17-18-19- 
20-21-22-23-24-25-26-27-28-29- 
30-31-32-33-34-35-36-37-38-39- 
40-41-42-43-44-45-46-47-48-49- 
50-51-52-53-54-55-56-57-58-59- 
60-61-62-63-64-65-66-67-68-69- 
70-71-72-73-74-75-76-77-78-79- 
80-81-82-83-84-85-86-87-88-89- 
90-91-92-93-94-95-96-97-98-99- 
100-101-102-103-104-105-106-107-108-109- 
110-111-112-113-114-115-116-117-118-119- 
120-121-122-123-124-125-126-127-128-129- 
130-131-132-133-134-135-136-137-138-139- 
140-141-142-143-144-145-146-147-148-149- 
150-151-152-153-154-155-156-157-158-159- 
160-161-162-163-164-165-166-167-168-169- 
170-171-172-173-174-175-176-177-178-179- 
180-181-182-183-184-185-186-187-188-189- 
190-191-192-193-194-195-196-197-198-199- 
200-201-202-203-204-205-206-207-208-209- 
210-211-212-213-214-215-216-217-218-219- 
220-221-222-223-224-225-226-227-228-229- 
230-231-232-233-234-235-236-237-238-239- 
240-241-242-243-244-245-246-247-248-249- 
250-251-252-253-254-255-256-257-258-259- 
260-261-262-263-26